一般训练流程

In [1]:
# 基本导入和设定
# Imports we need.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/data_generators/video_generated.py:34: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use("agg")


In [2]:
# 新建文件夹设定
DATA_DIR="t2t/data"
TMP_DIR="t2t/tmp"
TRAIN_DIR="t2t/train"
CHECKPOINT_DIR="t2t/checkpoints"
# Setup some directories
data_dir = DATA_DIR
tmp_dir = TMP_DIR
train_dir = TRAIN_DIR
checkpoint_dir = CHECKPOINT_DIR

tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)

In [6]:
# 读取问题和生成数据
# Fetch the librispeech_clean problem
image_mnist = problems.problem("image_mnist")
# The generate_data method of a problem will download data and process it into
# a standard format ready for training and evaluation.
image_mnist.generate_data(data_dir, tmp_dir)

INFO:tensorflow:Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to t2t/tmp/train-images-idx3-ubyte.gz
100% completed
INFO:tensorflow:Successfully downloaded train-images-idx3-ubyte.gz, 9912422 bytes.
INFO:tensorflow:Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to t2t/tmp/train-labels-idx1-ubyte.gz
113% completed
INFO:tensorflow:Successfully downloaded train-labels-idx1-ubyte.gz, 28881 bytes.
INFO:tensorflow:Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz to t2t/tmp/t10k-images-idx3-ubyte.gz
100% completed
INFO:tensorflow:Successfully downloaded t10k-images-idx3-ubyte.gz, 1648877 bytes.
INFO:tensorflow:Downloading http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz to t2t/tmp/t10k-labels-idx1-ubyte.gz
180% completed
INFO:tensorflow:Successfully downloaded t10k-labels-idx1-ubyte.gz, 4542 bytes.
INFO:tensorflow:Not downloading, file already found: t2t/tmp/train-images-idx3-ubyte.gz
INFO:tensorflow:Not downloa

In [7]:
# 定义模型
class MySimpleModel(t2t_model.T2TModel):

  def body(self, features):
    inputs = features["inputs"]
    filters = self.hparams.hidden_size
    h1 = tf.layers.conv2d(inputs, filters,
                          kernel_size=(5, 5), strides=(2, 2))
    h2 = tf.layers.conv2d(tf.nn.relu(h1), filters,
                          kernel_size=(5, 5), strides=(2, 2))
    return tf.layers.conv2d(tf.nn.relu(h2), filters,
                            kernel_size=(3, 3))

In [8]:
# 设定超参数和模型
hparams = trainer_lib.create_hparams("basic_1", data_dir=data_dir, problem_name="image_mnist")
hparams.hidden_size = 64
model = MySimpleModel(hparams, Modes.TRAIN)

INFO:tensorflow:Setting T2TModel mode to 'train'


In [10]:
# eager model 设定loss batchsiz和优化器
# Prepare for the training loop

# In Eager mode, opt.minimize must be passed a loss function wrapped with
# implicit_value_and_gradients
@tfe.implicit_value_and_gradients
def loss_fn(features):
  _, losses = model(features)
  return losses["training"]

# Setup the training data
BATCH_SIZE = 128
mnist_train_dataset = image_mnist.dataset(Modes.TRAIN, data_dir)
mnist_train_dataset = mnist_train_dataset.repeat(None).batch(BATCH_SIZE)

optimizer = tf.train.AdamOptimizer()

INFO:tensorflow:Reading data files from t2t/data/image_mnist-train*
INFO:tensorflow:partition: 0 num_data_files: 10
:::MLPv0.5.0 transformer 1541756288.884903431 (<ipython-input-10-ad383a5c9988>:13) input_order


In [11]:
# 训练
# Train
NUM_STEPS = 500

for count, example in enumerate(tfe.Iterator(mnist_train_dataset)):
  example["targets"] = tf.reshape(example["targets"], [BATCH_SIZE, 1, 1, 1])  # Make it 4D.
  loss, gv = loss_fn(example)
  optimizer.apply_gradients(gv)

  if count % 50 == 0:
    print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
  if count >= NUM_STEPS:
    break

:::MLPv0.5.0 transformer 1541756300.553159714 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
INFO:tensorflow:Transforming feature 'inputs' with image_modality.bottom
INFO:tensorflow:Transforming feature 'targets' with class_label_modality_10_64.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with class_label_modality_10_64.top
Step: 0, Loss: 8.216
:::MLPv0.5.0 transformer 1541756302.033714294 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756303.228427887 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756304.403623819 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 154175

:::MLPv0.5.0 transformer 1541756344.764834166 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756345.603878975 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756346.289660215 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
Step: 50, Loss: 0.938
:::MLPv0.5.0 transformer 1541756347.183411360 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756347.985276937 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756348.711599827 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_h

:::MLPv0.5.0 transformer 1541756386.349478006 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756387.201484442 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756388.078750372 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756388.974351168 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
Step: 100, Loss: 0.617
:::MLPv0.5.0 transformer 1541756389.705963373 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756390.599124908 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_

:::MLPv0.5.0 transformer 1541756429.435422659 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756430.209906101 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756430.920119762 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756431.994208336 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756432.968201160 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
Step: 150, Loss: 0.497
:::MLPv0.5.0 transformer 1541756433.997427225 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_

:::MLPv0.5.0 transformer 1541756471.155617952 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756471.891798258 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756472.820204735 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756473.759914398 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756474.515403032 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756475.325728416 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

...

:::MLPv0.5.0 transformer 1541756557.141865969 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756557.924324036 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756558.752695799 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756559.676681995 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756560.605500221 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756561.471573353 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

:::MLPv0.5.0 transformer 1541756601.236699820 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756602.023225546 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756602.919066906 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756603.833012342 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756604.604547262 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756605.841863394 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

:::MLPv0.5.0 transformer 1541756643.579966307 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756644.583052635 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756645.430710077 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756646.593009949 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756647.449205875 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756648.512996197 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

:::MLPv0.5.0 transformer 1541756686.458565950 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756687.414296150 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756688.297594309 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756689.534045696 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756690.351742506 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756691.138848305 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

:::MLPv0.5.0 transformer 1541756728.558925629 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756729.300191641 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756730.005394697 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756730.861428499 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756731.647534132 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756732.368499279 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

In [13]:
# 评估模型
model.set_mode(Modes.EVAL)
mnist_eval_dataset = image_mnist.dataset(Modes.EVAL, data_dir)

# Create eval metric accumulators for accuracy (ACC) and accuracy in
# top 5 (ACC_TOP5)
metrics_accum, metrics_result = metrics.create_eager_metrics(
    [metrics.Metrics.ACC, metrics.Metrics.ACC_TOP5])

for count, example in enumerate(tfe.Iterator(mnist_eval_dataset)):
  if count >= 200:
    break

  # Make the inputs and targets 4D
  example["inputs"] = tf.reshape(example["inputs"], [1, 28, 28, 1])
  example["targets"] = tf.reshape(example["targets"], [1, 1, 1, 1])

  # Call the model
  predictions, _ = model(example)

  # Compute and accumulate metrics
  metrics_accum(predictions, example["targets"])

# Print out the averaged metric values on the eval data
for name, val in metrics_result().items():
  print("%s: %.2f" % (name, val))

INFO:tensorflow:Reading data files from t2t/data/image_mnist-dev*
INFO:tensorflow:partition: 0 num_data_files: 1
:::MLPv0.5.0 transformer 1541756894.994960785 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756895.041650772 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756895.081145287 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756895.122818470 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756895.156726599 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756895.201843262 (/home

:::MLPv0.5.0 transformer 1541756897.136739016 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756897.179014444 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756897.223849058 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756897.268830776 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756897.309144735 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756897.350981236 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

:::MLPv0.5.0 transformer 1541756899.212550163 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756899.248612642 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756899.292002201 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756899.334664106 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756899.377775908 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756899.418421745 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

:::MLPv0.5.0 transformer 1541756901.184755802 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756901.212172985 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756901.239831209 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756901.266495705 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756901.297665834 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
:::MLPv0.5.0 transformer 1541756901.324448347 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.

:::MLPv0.5.0 transformer 1541756903.067457438 (/home/liuxinxin/.local/lib/python3.6/site-packages/tensor2tensor/utils/t2t_model.py:213) model_hp_initializer_gain: 1.5
accuracy: 0.98
accuracy_top5: 1.00
